# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Описание данных

сalls — количество звонков

minutes — суммарная длительность звонков в минутах

messages — количество sms-сообщений

mb_used — израсходованный интернет-трафик в Мб

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

## Изучим данные

In [1]:
#Подключим библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

#Откроем файл
df = pd.read_csv('/datasets/users_behavior.csv')

#Отобразим первые 5 строк таблицы
display(df.head(5))

##Просмотрим общую информации о таблице
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Вывод:

Таблица содержит 3214 строк и не имеет нулевых значений. В таблице 5 столбцов со следующим наименованием: calls, minutes, messages, mb_used, is_ultra. Столбец is_ultra в формате int64, все остальные столбцы в формате float64.

In [2]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

# Разделим данные на выборки

In [3]:
#Выделим признаки из общих данных. Это все столбцы за исключением столбца 'is_ultra'
features = df.drop(['is_ultra'], axis=1)

#Выделим целевой признак из общих данных. Это исключительно столбец 'is_ultra'.
target = df['is_ultra']

#Выделим 20% тестовых данных из всего объема данных
features_df1, features_test, target_df1, target_test = train_test_split(features, target, test_size=0.20, random_state=12345)

#Разделим отавшиеся 80% данных на тренировочные и валидационные(проверечное). Данные делим в пропорции: 75% от общего числа
#данных - это тренировочные, 25% от общего числа - это валидационные.
features_train, features_valid, target_train, target_valid = train_test_split(features_df1, target_df1, test_size=0.25, random_state=12345)

## Вывод:

Выделили признаки из общих данных, это все столбцы за исключением столбца 'is_ultra' и назвали их features. 
Выделили целевой признак из общих данных - это столбец 'is_ultra' и назвали его target.
Далее от общего массива данных отделили 20% тестовых данных. Оставшийся массив разделили на тренировочне данные - это 75% данных и валидационные - это 25% данных. В итоге данные разделились в пропорции 60/20/20.

In [4]:
print('Соотношение:')
print('--> тренировка', round(features_train.shape[0] / df.shape[0], 2))
print('--> валидация', round(features_valid.shape[0] / df.shape[0], 2))
print('--> тест', round(features_test.shape[0] / df.shape[0], 2))

Соотношение:
--> тренировка 0.6
--> валидация 0.2
--> тест 0.2


# Исследуем модели

## Модель 'дерево решений'

In [5]:
#Напишем цикл для глубины дерева решиня равное 9 (depth от 1 до 10) 
#Установим энтропию, вместо критерия Джини и random_state равное 12345, число признаков для выбора расщепления auto (max_features='auto')
print('Accuracy_score модели дерево решений на тестовой выборке:')
for depth in range(1,10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, max_features='auto', criterion = 'entropy')#установим гиперпараметры
    model.fit(features_train, target_train) #обучим модель на тренировочной выборке 
    predictions_valid = model.predict(features_valid) #предскажем результат на валидационной выборке
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid)) #оценим качество модели

Accuracy_score модели дерево решений на тестовой выборке:
max_depth = 1 : 0.7247278382581649
max_depth = 2 : 0.7325038880248833
max_depth = 3 : 0.7620528771384136
max_depth = 4 : 0.7542768273716952
max_depth = 5 : 0.7682737169517885
max_depth = 6 : 0.7807153965785381
max_depth = 7 : 0.7636080870917574
max_depth = 8 : 0.776049766718507
max_depth = 9 : 0.7698289269051322


## Вывод:

Для модели 'дерево решений' написали цикл с переборкой глубины от 1 до 9 и вывели список результатов. Для улучшения качества модели установили следующие гиперпараметры:
random_state=12345 
max_depth=depth (глубина дерева решений будет изменяться в диапозоне от 1 до 9)
max_features='auto' - число признаков для выбора расщепления
criterion = 'entropy' - изменен критерий Джини, который установлен по умолчания, на энтропию.
По полученным результатам можно сделать вывод, что максимальная доля правильных ответов модели наблюдается при глубине дерева равное 6 (max_depth = 6) и составляет 0.7807153965785381. 

## Модель 'случайный лес'

In [6]:
# Напишем функцию для модели 'случайный лес', чтобы выбрать лучшую модель в зависимости от количества деревьев.
best_model = None
best_result = 0
for est in range(1, 50):
    for depth in range(1,30):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est, criterion = 'entropy', min_samples_split=3, min_samples_leaf=1) #установим гиперпараметры
        model.fit(features_train, target_train) # обучим модель на тренировочной выборке
        result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
        if result > best_result:
            best_model = model
            best_result = result

print("Accuracy наилучшей модели 'случайный лес' на валидационной выборке:", best_result)

Accuracy наилучшей модели 'случайный лес' на валидационной выборке: 0.8087091757387247


## Вывод:

Для модели 'случайный лес' был написан двойной цикл с перебором количеством деревьев. Методом подбора максимального количества деревьев и глубины, пришли к выводу, что лучший результат наблюдается при количестве деревьев 30(n_estimators=30) и глубины 50(max_depth=50)  и составляет 0.8087091757387247. Для улучшения качества модели установили следующие гиперпараметры:
random_state=12345
max_depth=depth - глубина дерева (от 1 до 50)
n_estimators=est - количество деревьев в модели (от 1 до 30)
criterion = 'entropy' - изменен критерий Джини, который установлен по умолчания, на энтропию.
min_samples_split=3 - минимальное количество выборок, необходимое для разделения внутреннего узла.
min_samples_leaf=1 - минимальное количество образцов, которое должно быть в листовом узле.


## Модель 'логистическая регрессия'

In [7]:
model = LogisticRegression(random_state=12345)#создадим модель и установим гиперпараметры
model.fit(features_train, target_train) #обучим модель на тренировочной выборке
result = model.score(features_valid, target_valid) #посчитаем качество модели
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.6967340590979783


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Вывод:

Для модели 'логистическая регрессия' установили гиперпараметр random_state=12345 и получили долю правильных ответов модели равное accuracy =  0.6967340590979783

## Общий вывод:

Было обучено на тренировочной выборке и проверено 3 модели ('дерево решений', 'случайный лес', 'логистическая регрессия') на валидационной выборки. Лучший качество было достигнуто на модели 'случайный лес', здесь accuracy = 0.8087091757387247 при количестве деревьев равной 30 и глубине равной 50. 
На втором месте это 'дерево решений', здесь accuracy = 0.7807153965785381 при глубине дерева равное 6. 
На последнем месте 'логистическая регрессия', здесь accuracy = 0.6967340590979783

# Проверим модели на тестовой выборке

## Проверка модели 'дерево решений' на тестовой выборке

In [8]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6, max_features='auto', criterion = 'entropy') #установим гиперпараметры
model.fit(features_train, target_train)#обучим модель на тренировочной выборке 
predictions_test = model.predict(features_test)#предскажем результат на тестовой выборке
print('Accuracy_score модели дерево решений на тестовой выборке:', accuracy_score(target_test, predictions_test))#оценим качество модели

Accuracy_score модели дерево решений на тестовой выборке: 0.7916018662519441


## Проверка модели 'случайный лес' на тестовой выборке

In [9]:
model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=30, criterion = 'entropy', min_samples_split=3,min_samples_leaf=1)#установим гиперпараметры 
model.fit(features_train, target_train) #обучим модель на тренировочной выборке
result = model.score(features_test, target_test) #посчитаем качество модели на тестовой выборке
print("Accuracy наилучшей модели 'случайный лес' на тестовой выборке:", result)

Accuracy наилучшей модели 'случайный лес' на тестовой выборке: 0.7916018662519441


## Проверка модели 'логистическая регрессия' на тестовой выборке

In [10]:
model = LogisticRegression(random_state=12345, penalty='l2') #создадим модель и установим гиперпараметры
model.fit(features_train, target_train) #обучим модель на тренировочной выборке
result = model.score(features_test, target_test) #посчитаем качество модели
print("Accuracy модели логистической регрессии на тестовой выборке:", result)

Accuracy модели логистической регрессии на тестовой выборке: 0.702954898911353


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Вывод:

При проверке 3-х моделей на тестовой выборке доля правильных ответов у моделей 'дерево решений' и 'логистической регрессии' возросло, а у 'модели' случайный лес чуть уменьшилось. Самый высокая доля правильных ответов получилась у двух моделей - это "дерево решений" и "случайный лес", здесь доля правильных ответов одинаковая и составляет 0.7916018662519441, а у модели "логистическая регрессия" accuracy = 0.702954898911353.

# Проверка модели на адекватность

## Выделим из общего массива данных только строки с тарифом ultra.

In [11]:
#Выделим из общего массива данных только строки с тарифом ultra и поместим эти данные в таблицу df_test2
df_test2 = df.query('is_ultra == 1')

#Отобразим первые 5 строк таблицы df_test2
display(df_test2.head(5))

#Отобразим информацию о таблице df_test2
df_test2.info()

,calls,minutes,messages,mb_used,is_ultra
3,106.0,745.53,81.0,8437.39,1
6,57.0,431.64,20.0,3738.90,1
8,7.0,43.39,3.0,2538.67,1
10,82.0,560.51,20.0,9619.53,1
14,108.0,587.90,0.0,14406.50,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 985 entries, 3 to 3213
Data columns (total 5 columns):
calls       985 non-null float64
minutes     985 non-null float64
messages    985 non-null float64
mb_used     985 non-null float64
is_ultra    985 non-null int64
dtypes: float64(4), int64(1)
memory usage: 46.2 KB


## Создадим заведомо некорректную таблицу. В столбце 'is_ultra' заменим 1 на 0. 

In [12]:
df_zero = df_test2.replace(1,0)
display(df_zero.head(5))

,calls,minutes,messages,mb_used,is_ultra
3,106.0,745.53,81.0,8437.39,0
6,57.0,431.64,20.0,3738.90,0
8,7.0,43.39,3.0,2538.67,0
10,82.0,560.51,20.0,9619.53,0
14,108.0,587.90,0.0,14406.50,0


## Выделим признаки и целевой признак из некорректных данных таблицы df_zero

In [13]:
features_test_zero = df_zero.drop(['is_ultra'], axis=1)
target_test_zero = df_zero['is_ultra']

## Проверим 3 модели на адекватность на некорректных данных

### Проверка модели 'дерево решений' на адекватность

In [14]:
model = DecisionTreeClassifier(random_state=12345, max_depth=depth, max_features='auto', criterion = 'entropy')
model.fit(features_train, target_train)
predictions_test = model.predict(features_test_zero) #предскажем результат на некорректной выборке
print('Accuracy_score модели дерево решений на некорректной выборке:', accuracy_score(target_test_zero, predictions_test)) 

Accuracy_score модели дерево решений на некорректной выборке: 0.1918781725888325


###  Проверка модели 'случайный лес' на адекватность

In [15]:
model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=30, criterion = 'entropy', min_samples_split=3,min_samples_leaf=1) 
model.fit(features_train, target_train) 
result = model.score(features_test_zero, target_test_zero) # посчитаем качество модели на некорректной выборке
print("Accuracy наилучшей модели на некорректной выборке:", result)

Accuracy наилучшей модели на некорректной выборке: 0.19289340101522842


###  Проверка модели 'логистическая регрессия' на адекватность

In [16]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train) 
result = model.score(features_test_zero, target_test_zero) 
print("Accuracy модели логистической регрессии на некорректной выборке:", result)

Accuracy модели логистической регрессии на некорректной выборке: 0.9532994923857868


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Вывод:

Для проберки адекватности моделей, был сделан срез данных только по тарифу ultra, т.е. в столбце is_ultra все значения равнялись еденице. Для того чтобы проверить адекватность моделей еденицы были заменены на нули, таким образом мы создали срез не корректных данных. Далее протестировали все три модели на некоректной выборке. Т.е. указали вместо тестовой выборки, выборку некорректную. 
Модель 'дерево решений' при глубине max_dept=6 ведет себя адекватно, т.е. количество правильных ответов становится равной 0.4436548223350254.  
Модель 'случайны лес' ведет себя адекватно и показывает accuracy = 0.19289340101522842.
Модель 'логистическая регрессия' ведет себя не адекватно и показывает высокий accuracy = 0.9532994923857868

In [17]:
target_test.value_counts() / features_test.shape[0]

0    0.695179
1    0.304821
Name: is_ultra, dtype: float64

In [18]:
best_model_tree = DecisionTreeClassifier(random_state = 500, max_depth = 7)
best_model_tree.fit(features_train, target_train) # обучаем на тренировке

# сформируем предсказания на тесте
prob = best_model_tree.predict_proba(features_test)[:, 1]

print('AUC best_tree test', roc_auc_score(target_test, prob))
print('Гиперпараметры forest_rev', best_model_tree)

AUC best_tree test 0.728147970597635
Гиперпараметры forest_rev DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=500, splitter='best')


# Чек-лист готовности проекта

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
